In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

from model import UniSkip, Encoder
from data_loader import DataLoader
from vocab import load_dictionary
from config import *
from torch import nn
import numpy as np

from torch.autograd import Variable
import torch

class UsableEncoder:
    
    def __init__(self, loc="../../dir_HugeFiles/prev_model/skip-best-loss10.237"):
        print("Preparing the DataLoader. Loading the word dictionary")
        self.d = DataLoader(sentences=[''], word_dict=load_dictionary('./data/dummy_corpus.txt.pkl'))
        self.encoder = None
        
        print("Loading encoder from the saved model at {}".format(loc))
        model = UniSkip()
        model.load_state_dict(torch.load(loc, map_location=lambda storage, loc: storage))
        self.encoder = model.encoder
        if USE_CUDA:
            self.encoder.cuda(CUDA_DEVICE)
    
    def encode(self, text):
        def chunks(l, n):
            """Yield successive n-sized chunks from l."""
            for i in range(0, len(l), n):
                yield l[i:i + n]
        
        ret = []
        
        for chunk in chunks(text, 100):
            print("encoding chunk of size {}".format(len(chunk)))
            indices = [self.d.convert_sentence_to_indices(sentence) for sentence in chunk]
            indices = torch.stack(indices)
            indices, _ = self.encoder(indices)
            indices = indices.view(-1, self.encoder.thought_size)
            indices = indices.data.cpu().numpy()
            
            ret.extend(indices)
        ret = np.array(ret)
        
        return ret

usable_encoder = UsableEncoder()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs
Preparing the DataLoader. Loading the word dictionary
Making reverse dictionary
Loading encoder from the saved model at ../../dir_HugeFiles/prev_model/skip-best-loss10.237


### get features from encoder

In [2]:
text = ['good morning !', 'i miss you peter']
# text should be a list of list
features = usable_encoder.encode(text)

encoding chunk of size 2
time: 49.9 ms


In [3]:
features

array([[-2.55208015e-01,  8.36868057e-05, -1.79179654e-01, ...,
        -1.93723757e-02,  4.73951437e-02,  1.64058548e-03],
       [-3.00997555e-01,  1.33039244e-03, -1.06062576e-01, ...,
        -3.13864164e-02,  1.53462544e-01,  7.02177808e-02]], dtype=float32)

time: 29.5 ms
